<a href="https://colab.research.google.com/github/shaluduggal/gcp_paper/blob/machinelearning/brest_tumor_25_mar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install visualkeras
!pip install tensorflow
!pip install keras-tuner

In [4]:
#from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, Callback
from keras.optimizers import Adam
import tensorflow as tf
from tensorflow import keras
#from tensorflow.keras.preprocessing.image import ImageDataGenerator
#import visualkeras
from keras.utils import plot_model
import math
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters
from keras.layers import LeakyReLU,   Conv2D
# CNN Architecture Visualization Library
from keras.layers import Flatten,Dense,Dropout
from keras import optimizers
from keras.layers import BatchNormalization
import keras_tuner
import cv2
from PIL import Image
from keras.layers import  MaxPool2D,MaxPooling2D, Activation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import seaborn as sns
from PIL import Image ,ImageOps
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import MaxPooling2D
from keras.layers import Conv2D
from keras import layers
import tensorflow as tf

In [3]:
from google.colab import drive
drive.mount('/content/drive')
from pathlib import Path
benign1 =[]
frames= Path('/content/drive/MyDrive/dataset/benign')
for file in frames.glob('*.png'):
    img=Image.open(file)
    img=ImageOps.grayscale(img)    #convert to grey scale
    img=img.resize((224,224))
    #we can use img.thumnail () to resize so that aspect ratio is maintained
  #  img.save('D:/research_p/dataset/benign'+file+".jpg")
    benign1.append(np.array(img))
b =len(benign1)
malignant1 =[]
frames= Path('/content/drive/MyDrive/dataset/malignant')
for file in frames.glob('*.png'):
    img=Image.open(file)
    img=ImageOps.grayscale(img)    #convert to grey scale
    img=img.resize((224,224))
   # img.save('D:/research_p/dataset/malignant')
    malignant1.append(np.array(img))
m =len(malignant1)
normal1 =[]
frames= Path('/content/drive/MyDrive/dataset/normal')
for file in frames.glob('*.png'):
    img=Image.open(file)
    img=ImageOps.grayscale(img)     #convert to grey scale
    img=img.resize((224,224))
    #img.save('D:/research_p/dataset/normal')
    normal1.append(np.array(img))
n =len(normal1)
print(n)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


NameError: name 'Image' is not defined

In [ ]:
#Features combination
#combining data into images
images =[ ]
images.extend(malignant1)
images.extend(benign1)
images.extend(normal1)
#en(images)
#images[203].shape
#adding labels
labels=[ ]
m=[1]*m
b=[2]*b
n=[0]*n
labels.extend(m)
labels.extend(b)
labels.extend(n)
len(labels)
print(len(b)+len(m)+len(n))

In [4]:
def normalize(x):
    min_val = np.min(x)
    max_val = np.max(x)
    x = (x-min_val) / (max_val-min_val)
    return x
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test =train_test_split(images,labels,test_size=0.2,random_state=41)
x_train=np.array(x_train)
x_test=np.array(x_test)
y_train=np.array(y_train)
y_test=np.array(y_test)
#x_train = normalize(x_train)
#x_test = normalize(x_test)
x_train=x_train.reshape((624,224,224,1))
x_test=x_test.reshape((156,224,224,1))
y_train = keras.utils.to_categorical(y_train,num_classes=3)
y_test = keras.utils.to_categorical(y_test,num_classes=3)

#print(x_train.shape)
#print(x_test.shape)
#input_shape1=(x_test[0].shape)
#print(input_shape1)
#print(y_train.shape,y_test[0].shape)

NameError: name 'images' is not defined

In [ ]:
#basic model1
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import MaxPooling2D
from keras.layers import Conv2D
from keras import layers
import tensorflow as tf
model = Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(Flatten())
model.add(layers.Dense(128, activation='relu'))
#model.add(layers.Dropout(rate=0.3))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(3,activation='softmax'))
model.summary()
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
checkpoint_path = "/bin/chk_point1"
#checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = [EarlyStopping(monitor='val_loss', patience=2),
              keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)]

history = model.fit(x_train, y_train, epochs=10, validation_data=(x_test,y_test),callbacks=[cp_callback])

In [14]:
#Saving the about model 1 trained parameters
import pickle

# save the iris classification model a_s a pickle file
model_pkl_file1 = "First_trained_model_25-03-2024"

with open(model_pkl_file1, 'wb') as file:
    pickle.dump(model, file)
#with open(model_pkl_file1, 'rb') as file:
    model = pickle.load(file)

# evaluate model
#y_predict = model.predict(x_test)

# check results
#print(classification_report(y_test, y_predict))
#

5/5 [==============================] - 5s 1s/step


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 8))
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')
plt.savefig("output_report.png")

In [5]:
#model2
model = Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(64, (3, 3), activation='tanh'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(128, (3, 3), activation='tanh'))
model.add(layers.BatchNormalization())
model.add(Flatten())
model.add(layers.Dense(128, activation='tanh'))
#model.add(layers.Dropout(rate=0.3))
model.add(layers.Dense(64, activation='tanh'))
model.add(layers.Dense(3,activation='softmax'))
model.summary()
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
checkpoint_path = "/bin/chk_point1_25_mar"
#checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = [EarlyStopping(monitor='val_loss', patience=2),
              keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)]

history = model.fit(x_train, y_train, epochs=10, validation_data=(x_test,y_test),callbacks=[cp_callback])

NameError: name 'Sequential' is not defined

In [1]:
#hyper tuning optimizer, model2 for activation tanh
#optimizing optimizer
def build_model(hp):  # random search passes this hyperparameter() object
  model= Sequential()
  model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 1)))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(Flatten())
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(3,activation="softmax"))
  model.summary()
  optimizer=hp.Choice("otimizers",values=["adam","rmsprop","sgd","adadelta"])
  model.compile(optimizer=optimizer,  metrics=['accuracy'], loss="sparse_categorical_crossentropy")
  return model
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # how many model variations to test?
    executions_per_trial=3,  # how many trials per variation? (same model could perform differently)
    directory="/bin/dir_1/optimizer_model_data",
    project_name='breast_tumor_paper1'
)
tuner.search(x_train,y_train,epochs=2,validation_data=(x_test,y_test))
print(tuner.get_best_hyperparameters()[0].values)
model=tuner.get_best_models(num_models=1)[0]
model.summary()
model.fit(x_train,y_train,batch_size=32,epochs=25,initial_epoch=6,validation_data=(x_test,y_test))
history = model.fit(x_train, y_train, epochs=10, validation_split=0.1)
# save the iris classification model as a pickle file
import pickle
model_pkl_file2 = "Seconf_trained_model_25_mar"
with open(model_pkl_file2, 'wb') as file:
    pickle.dump(model, file)


NameError: name 'RandomSearch' is not defined

In [27]:
#model 3 with keras tuner activation relu
def build_model(hp):  # random search passes this hyperparameter() object
  model = keras.models.Sequential()
  model.add(Conv2D(
           filters=hp.Int('conv_filters_layer1',min_value=32,max_value=128,step=16),
           kernel_size=hp.Choice('conv_kernel_layer1',values=[3,4]),
           input_shape=(224,224,1)
    ))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  for i in range(hp.Int('n_layers', 1, 4)):  # adding variation of layers.
        model.add(Conv2D(hp.Int(f'conv_{i}_units',
                                min_value=32,
                                max_value=256,
                                step=32), (3, 3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Flatten())
  for i in range(hp.Int('n_connections', 1, 4)):
        model.add(Dense(hp.Choice(f'n_nodes',values=[128, 256, 512])))
        model.add(Activation('relu'))
  if hp.Boolean(name="dropout"):
      model.add(Dropout(rate=0.5))
  model.add(Dense(3))
  model.add(Activation("softmax"))
  model.compile(optimizer="adam",
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])

  return model

In [ ]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # how many model variations to test?
    executions_per_trial=3,  # how many trials per variation? (same model could perform differently)
    directory="/bin/dir_1/optimizer_model_data",
    project_name='breast_tumor_paper1'
)
tuner.search(x_train,y_train,epochs=2,validation_data=(x_test,y_test))
print(tuner.get_best_hyperparameters()[0].values)
model=tuner.get_best_models(num_models=1)[0]
model.summary()
model.fit(x_train,y_train,batch_size=32,epochs=25,initial_epoch=6,validation_data=(x_test,y_test))

In [ ]:
import pickle
# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
# save the iris classification model as a pickle file
model_pkl_file2 = "third_ optimzer_trained_model_25_mar"
with open(model_pkl_file2, 'wb') as file:
    pickle.dump(model, file)

In [ ]:
models = tuner.get_best_models(num_models=2)
best_model = models[0]
print(best_hps)

# Hyperparameter tuning